### Deploying an Huggingface Model in Azure

In [ ]:
# conda create -n huggingface python=3.12
# pip install transformers
# pip install azure-ai-ml
# pip install torch

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#### 1. Load the huggingface model/ save locally

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save locally
# model.save_pretrained("./model")
# tokenizer.save_pretrained("./model")

#### 2. Connect to Azure

In [41]:
from azure.ai.ml.entities import Model
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# Connect to Azure ML workspace
ml_client = MLClient(
    credential,
    subscription_id="b4d33442-64f0-407f-bf53-dd917fd40186",
    resource_group_name="UOS-ICT-TST-DATA-RG",
    workspace_name="sam-ml",
)



Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


#### 3. Register the model

In [ ]:
# Register the model
model = ml_client.models.create_or_update(
    Model(
        name="distilbert-sentiment",
        path="./model",
        description="Hugging Face DistilBERT sentiment analysis model",
    )
)
print(f"Registered model: {model.name} with version: {model.version}")

#### 4. Create an azure environment

In [ ]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="huggingface-env",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file={
        "channels": ["defaults", "conda-forge"],
        "dependencies": [
            "python=3.12",
            "pip",
            {"pip": ["transformers", "torch", "azure-ai-ml"]},
        ],
    },
)
ml_client.environments.create_or_update(env)

#### 5. Create an azure endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration

# Create endpoint
endpoint = ManagedOnlineEndpoint(
    name="huggingface-endpoint",
    auth_mode="key",
)
ml_client.online_endpoints.begin_create_or_update(endpoint)

#### 6. Create an deployment

In [ ]:
# create a score.py with inference info 
# Create deployment
deployment = ManagedOnlineDeployment(
    name="huggingface-deployment",
    endpoint_name=endpoint.name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./model/", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(deployment)



In [ ]:
# Set the deployment as default
endpoint.traffic = {"huggingface-deployment": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

#### 7. Test 

In [ ]:
import requests

endpoint_url = "https://huggingface-endpoint.australiaeast.inference.ml.azure.com/score"
api_key = "<your-api-key>" 

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
data = {"text": ["I love this product!", "This is terrible."]}

response = requests.post(endpoint_url, headers=headers, json=data)
print(response.json())